In [1]:
import urllib3
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import numpy as np
from google.colab import drive
import os

drive.mount('/content/drive')
http = urllib3.PoolManager()
directory ='/content/drive/MyDrive/Assignment-2/'

Mounted at /content/drive


In [2]:
def delUnwanted (res):
  del res['purchaseOptions']
  del res['dbTime']
  del res['ephemeraFlag']
  del res['allowComments']
  del res['getProgramTermsAccepted']
  del res['htmlAbstractLink']
  del res['htmlLink']
  del res['html_flag']
  del res['isCustomDenial']
  del res['isDynamicHtml']
  del res['isEarlyAccess']
  del res['isEphemera']
  del res['isFreeDocument']
  del res['isGetAddressInfoCaptured']
  del res['isGetArticle']
  del res['isMarketingOptIn']
  del res['isMorganClaypool']
  del res['isNotDynamicOrStatic']
  del res['isNow']
  del res['isNumber']
  del res['isOUP']
  del res['isOpenAccess']
  del res['isProduct']
  del res['isPromo']
  del res['isReadingRoomArticle']
  del res['isSAE']
  del res['isSMPTE']
  del res['isStandard']
  del res['isStaticHtml']
  del res['mediaPath']
  del res['mlTime']
  del res['ml_html_flag']
  del res['openAccessFlag']
  del res['pdfPath']
  del res['pdfUrl']
  del res['persistentLink']
  del res['pubLink']
  del res['rightsLink']
  del res['rightsLinkFlag']
  del res['sections']
  del res['sourcePdf']
  del res['userInfo']
  return res

In [3]:
def consistData(res):
  
  kes = np.array(list(res.keys()))
  new_keys = np.intersect1d(kes, keys_to_consider)
  new_res = {x:res[x] for x in new_keys}
  return new_res

In [5]:
def rearrangeData(res, sponsorsFlag = True):
  issn = {value['format'] + '_value':value['value'] for value in res['issn']}
  del res['issn']
  res.update(issn)

  metrics = {key + "_metrics":value for key, value in res['metrics'].items()}
  del res['metrics']
  res.update(metrics)

  pubTopics = [x['name'] for x in res['pubTopics']]
  del res['pubTopics']
  res.update({'pubTopics': pubTopics})

  if sponsorsFlag:
    sponsors_name = [x['name'] for x in res['sponsors']]
    sponsors_url = [x['url'] for x in res['sponsors']]
    del res['sponsors']
    res.update({'sponsors_name': sponsors_name})
    res.update({'sponsors_url': sponsors_url})

  return res

In [6]:
def getJson(url, http):
  r = http.request('GET', url)
  soup = BeautifulSoup(r.data,features="html.parser")
  da = soup.find_all('script', type='text/javascript')
  x = re.search("xplGlobal\.document\.metadata=.*", str(da[8]))
  res = json.loads(x.group()[28:-1])
  return res

In [7]:
def one_NF (df, col_name, with_split = True):
  if with_split == True:
    df[col_name] = df[col_name].map(lambda x: x[0].split(','))
  return pd.DataFrame({col:np.repeat(df[col].values, df[col_name].str.len()) for col in df.columns.difference([col_name])}).assign(**{col_name:np.concatenate(df[col_name].values)})[df.columns.tolist()]

In [8]:
df_articles = pd.DataFrame()
df_authors = pd.DataFrame()
df_keywords = pd.DataFrame()
df_sponsors = pd.DataFrame()
df_pubTopics = pd.DataFrame()
keys_to_consider = delUnwanted (getJson('https://ieeexplore.ieee.org/document/6661315/', http))
keys_to_consider = np.array(list(keys_to_consider.keys()))

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
def getAuthorDf(res):
  df = pd.DataFrame(res['authors'])
  try:
    df.drop('bio', axis=1, inplace=True)
  except:
    print("no bio")
  try:
    new_df = one_NF(df, 'affiliation')
  except:
    new_df = df
    new_df['affiliation'] = np.nan
  del res['authors']
  new_df['articleId'] = res['articleId']
  return (new_df, res)

In [10]:
def getKeywordsDf(res):
  df_keywords = pd.DataFrame(res['keywords'])
  df_keywords_new = one_NF (df_keywords, 'kwd', with_split=False)
  del res['keywords']
  df_keywords_new['articleId'] = res['articleId']
  return (df_keywords_new, res)

In [11]:
def getArticleDf (res, sponsorsFlag = True):
  df_pubTopics = pd.DataFrame({'pubTopics': res['pubTopics']})
  df_pubTopics['articleId'] = res['articleId']
  del res['pubTopics']
  if sponsorsFlag:
    df_sponsors = pd.DataFrame({'sponsors_name': res['sponsors_name'], 'sponsors_url': res['sponsors_url']})
    df_sponsors['articleId'] = res['articleId']
    del res['sponsors_name']
    del res['sponsors_url']
    df = pd.DataFrame.from_records([res])
    df.drop_duplicates(inplace=True)
    return (df, df_pubTopics, df_sponsors)
  df = pd.DataFrame.from_records([res])
  df.drop_duplicates(inplace=True)
  return (df, df_pubTopics)

In [12]:
remaining_files = os.listdir(directory)
file_num=0
finished_urls = []

In [ ]:
for filename in remaining_files:
  print(filename)
  print("File "+ str(file_num) + " / " + str(len(os.listdir(directory))))
  df = pd.read_excel(directory + '/' + filename)
  urls = list(df['URL'])
  record_num = 0
  for url in urls:
    if url not in finished_urls:
      print(url)
      record_num += 1
      print("Record "+ str(record_num) + " / " + str(len(urls)))
      sponsorsFlag = True
      res = getJson(url, http)
      res = consistData(res)
      try:
        print(res['sponsors'])
      except:
        sponsorsFlag = False

      
      df_author, res = getAuthorDf(res)
      
      df_keyword, res = getKeywordsDf(res)

      res = rearrangeData(res, sponsorsFlag)
      if sponsorsFlag:
        df_article, df_pubTopic, df_sponsor = getArticleDf (res, sponsorsFlag)
      else:
        df_article, df_pubTopic = getArticleDf (res, sponsorsFlag)

      df_authors = df_authors.append(df_author, ignore_index=True)
      df_keywords = df_keywords.append(df_keyword, ignore_index=True)
      df_articles = df_articles.append(df_article, ignore_index=True)
      df_pubTopics = df_pubTopics.append(df_pubTopic, ignore_index=True)
      if sponsorsFlag:
        df_sponsors = df_sponsors.append(df_sponsor, ignore_index=True)
      finished_urls.append(url)
  file_num += 1
  remaining_files.remove(filename)

**Saving Files to my Google drive**

In [49]:
df_authors.to_csv ('/content/drive/MyDrive/Assignment-2/Data/df_authors.csv', index=False)
df_keywords.to_csv('/content/drive/MyDrive/Assignment-2/Data/df_keywords.csv', index=False)
df_articles.to_csv('/content/drive/MyDrive/Assignment-2/Data/df_articles.csv', index=False)
df_articles.to_csv('/content/drive/MyDrive/Assignment-2/Data/df_articles.csv', index=False)
df_pubTopics.to_csv('/content/drive/MyDrive/Assignment-2/Data/df_pubTopics.csv', index=False)
df_sponsors.to_csv('/content/drive/MyDrive/Assignment-2/Data/df_sponsors.csv', index=False)